In [17]:
path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/seekfold_phagedepo"
import pandas as pd
import os 

path_info = f"/media/concha-eloko/Linux/depolymerase_building/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)
dico_folds = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]
header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]
depo_results = {}
for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    for _,row in results_df.iterrows() :
        fold = info_df[info_df["ID"] == row["target"]]["X_Group_Name"].values[0]
        if fold != "jelly-roll" :
            if row["prob"] >= 0.5 :
                depo_results[row["query"].split(".pdb")[0]] = {"Fold" : fold , "Prob" : row["prob"]}
            elif fold == "right-handed beta-helix" and row["prob"] >= 0.2 :
                depo_results[row["query"].split(".pdb")[0]] = {"Fold" : fold , "Prob" : row["prob"]}
                


In [23]:
len(depo_results)


936

In [26]:
import subprocess
import os

path_project = "/media/concha-eloko/Linux/depolymerase_building"
path_pdb = f"{path_project}/sword2_PhageDepo"
path_db = f"/media/concha-eloko/Linux/depolymerase_building/RefDepo_domains/RefDepo_domains_db"
path_tmp = f"{path_project}/tmp"

paths_domain = []
swords_again = []
for file in depo_results :
    if os.path.isdir(f"{path_pdb}/{file}/{file}_A/Protein_Units")==False : 
        swords_again.append(file)
    else :
        n=0
        for file_2 in os.listdir(f"{path_pdb}/{file}/{file}_A/Protein_Units") :
            if file_2[-3:] == "pdb" :
                n+=1
                path = f"{path_pdb}/{file}/{file}_A/Protein_Units/{file_2}"
                paths_domain.append(path)
        if n==0 :
            swords_again.append(file)


In [21]:
len(swords_again)

611

In [27]:
done_prot = []
for prot in depo_results :
    if prot not in swords_again :
        done_prot.append(prot)
        
import pprint
pp = pprint.PrettyPrinter(width = 200, sort_dicts = True, compact = True)
pp.pprint(swords_again)

['5416', '4603', '6115', '1809', '5282', '644', '6321', '6339', '3338', '1343', '5137', '5474', '2063', '6058', '5505', '5567', '5641', '636', '4258', '6707', '3461', '5468', '5226', '5081', '3354',
 '326', '725', '5136', '5651', '2761', '3864', '3237', '3706', '1122', '5777', '1064', '289', '629', '518', '5418', '4467', '5126', '536', '5472', '4911', '2417', '505', '3352', '4250', '11', '764',
 '2591', '5510', '5623', '5495', '4072', '2788', '6466', '2702', '6225', '5603', '683', '6314', '643', '4597', '4296', '5590', '5053', '5172', '977', '6132', '6763', '758', '1806', '5724', '5024',
 '5197', '1450', '868', '3077', '5036', '654', '4634', '5055', '201', '4229', '1721', '3944', '5790', '2128', '6439', '3151', '4736', '2916', '6453', '5917', '1535', '5916', '6361', '722', '1805',
 '4704', '4602', '5343', '6065', '3207', '659', '6514', '4894', '5386', '651', '503', '3519', '4632', '4110', '713', '4576', '607', '5045', '6447', '2884', '4706', '4670', '785', '4165', '1013',
 '5734', '527

In [ ]:
import pprint
pp = pprint.PrettyPrinter(width = 200, sort_dicts = True, compact = True)
pp.pprint(list(depo_results.keys()))


In [9]:
import os 
import subprocess
from multiprocessing.pool import ThreadPool
import ast
import random

path_project = f"/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023"
path_work = "/home/conchae/PhageDepo_pdb"
path_pdb = f"{path_work}/phagedepo_out"
path_out = f"{path_work}/sword2_PhageDepo_pt2"


with open(f"{path_project}/depolymerase_proteins.PhageDepo.txt", "r") as file:
    content = file.read()
    to_predic = ast.literal_eval(content)

def sword2_pred(pdb_file) :
    """
    This function makes a sword prediction from a pdb_file.
    It takes as an input a pdb file, and outputs swords2 files.     
    The path_out is to be adjusted to the context.
    """
    with open(f"{path_out}/sword2.log" ,"a+") as outfile :
        path_sword = "/home/conchae/software/SWORD2"
        name = pdb_file.split("/")[-1].split(".pdb")[0]
        if os.path.isdir(f"{path_out}/{name}")==False :
            print("The outdir does not exsit, let's see what's up")
            sword_command = f" {path_sword}/SWORD2.py -i {pdb_file} -o {path_out}/{name} -x 5"
            sword_subprocess = subprocess.Popen (sword_command , shell = True, stdout = subprocess.PIPE, stderr = subprocess.STDOUT)
            hmm_out, hmm_err = sword_subprocess.communicate()
            print(hmm_out)
            outfile.write(f"{name} is done.")
    
done_protein = ['3777', '1856', '2638', '1533', '3737', '1998', '3954', '2267', '1020', '202', '1436', '2800', '1502', '16', '4299', '3463', '2617', '1848', '1569', '2715', '397', '4042', '1620', '4088', '1857',
 '1858', '364', '1695', '1107', '3036', '3863', '1813', '4071', '396', '3792', '3274', '3425', '3456', '1755', '2624', '1779', '2944', '1424', '1106', '1091', '2038', '4143', '2963', '3947', '3940',
 '3754', '154', '3201', '1493', '2137', '1022', '2217', '4221', '2216', '3074', '4178', '3955', '2747', '3379', '3801', '2691', '4247', '4227', '3965', '2011', '3498', '2972', '402', '3442', '2540',
 '1503', '3296', '1517', '4248', '1420', '1717', '333', '403', '2531', '2962', '4477', '1203', '2700', '1516', '2596', '2732', '3720', '4211', '1965', '2709', '3651', '218', '1774', '369', '3174',
 '3909', '3930', '4338', '3767', '1966', '334', '1985', '4180', '4210', '4319', '3727', '29', '1531', '2714', '3683', '1918', '2632', '1842', '1344', '1463', '2243', '3799', '1727', '2956', '3126',
 '3603', '1994', '2602', '4472', '1143', '4302', '2908', '183', '2066', '1645', '4446', '4475', '1431', '2062', '2941', '1855', '3459', '1470', '2191', '226', '4261', '221', '3939', '2597', '2973',
 '4259', '2089', '4090', '2587', '428', '4230', '2464', '4120', '448', '294', '141', '3477', '2778', '3275', '3334', '2093', '3085', '4502', '3374', '2661', '2751', '134', '167', '2934', '1515',
 '1513', '1110', '1571', '1840', '2782', '3210', '2068', '3953', '3229', '2793', '3010', '3854', '2603', '159', '3113', '2892', '4549', '1570', '2291', '1847', '3684', '1147', '2598', '3208', '1568',
 '3239', '3317', '4497', '4089', '1454', '1527', '3329', '3392', '1952', '3056', '1566', '2012', '3800', '1536', '260', '2486', '361', '405', '2899', '2945', '3398', '4481', '1658', '2912', '1035',
 '1789', '3054', '3705', '1501', '2589', '3091', '4489', '3813', '3782', '2754', '4086', '2480', '3133', '425', '2765', '3532', '445', '1163', '2796', '3779', '144', '4036', '3906', '331', '4417',
 '3698', '4358', '1095', '1111', '3071', '1214', '3309', '3957', '1810', '2660', '3357', '1437', '1532', '3303', '4171', '3355', '1996', '164', '1112', '146', '3735', '1290', '3282', '2703', '4136',
 '3570', '3840', '4511', '4500', '184', '2749', '17', '3798', '1108', '3776', '163', '347', '1790', '2816', '3070', '373', '408', '4044', '2034', '2590', '4385', '3728', '3732', '3176', '3778',
 '1477', '2713', '1065', '2927', '2891', '1492', '233', '1661', '1923', '4186', '4160', '3276', '3164', '3597', '1563', '3595', '4215', '3112', '4070', '2002', '1869', '3750', '1711', '4263', '225']

paths = [f"{path_pdb}/{file}.pdb" for file in to_predic if file not in done_protein]

for path in random.sample(paths, len(paths)) :
    sword2_pred(path)

In [16]:
# Run the scabtch as soon as the PPT command is over :

import os 
import subprocess
from multiprocessing.pool import ThreadPool
import ast
import random

path_work = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher"
path_project = f"{path_work}/ficheros_28032023"


sh_file = f"{path_project}/script_files/sword2.parralel.org.PhageDepo.sh"
sh_file_txt = open(sh_file).read()
for file in os.listdir(f"{path_project}/script_files") :
    if os.path.isdir(f"{path_project}/script_files/{file}") == True :
        with open(f"{path_project}/script_files/{file}/script_{file}.PhageDepo.sh","w") as outfile :
            outfile.write(sh_file_txt + "\n")
            outfile.write(f"python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/script_files/sword2_parralel.org.PhageDepo.py")

for n in range(1,30) :
    if n != 11 :
        os.chdir(f"{path_project}/script_files/SWORD{n}/")
        os.system(f"pwd")
        os.system(f"sbatch {path_project}/script_files/SWORD{n}/script_SWORD{n}.PhageDepo.sh")
           
for rep in os.listdir(f"{path_project}/sword2_Minibatch_pt2") :
    for file in os.listdir(f"{path_project}/sword2_Minibatch_pt2") :
        if os.path.isdir(f"{path_project}/sword2_Minibatch_pt2/{file}/{file}_A/SWORD") == False :
            os.system(f"rm -rf {path_project}/sword2_Minibatch_pt2/{file}")
            print(file)

['5416', '4603', '6115', '1809', '5282', '644', '6321', '6339', '3338', '1343', '5137', '5474', '2063', '6058', '5505',
 '5567', '5641', '636', '4258', '6707', '3461', '5468', '5226', '5081', '3354', '326', '725', '5136', '5651', '2761', '3864',
 '3237', '3706', '1122', '5777', '1064', '289', '629', '518', '5418', '4467', '5126', '536', '5472', '4911', '2417', '505',
 '3352', '4250', '11', '764', '2591', '5510', '5623', '5495', '4072', '2788', '6466', '2702', '6225', '5603', '683', '6314',
 '643', '4296', '5590', '5053', '5172', '977', '6132', '6763', '758', '1806', '5724', '5024', '5197', '1450', '868', '3077',
 '5036', '654', '4634', '5055', '201', '4229', '1721', '3944', '5790', '2128', '6439', '3151', '4736', '2916', '6453',
 '5917', '1535', '5916', '6361', '722', '1805', '4704', '4602', '5343', '6065', '3207', '659', '6514', '4894', '5386', '651',
 '503', '3519', '4632', '4110', '713', '607', '5045', '6447', '2884', '4706', '4670', '785', '4165', '1013', '5734', '5278',
 '4151', '

In [ ]:
***
# Repeat the missing SWORD2 predictions :
import os 
import subprocess
from multiprocessing.pool import ThreadPool
import ast
import random


path_project = f"/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023"
path_work = "/home/conchae/PhageDepo_pdb"
path_pdb = f"{path_work}/phagedepo_out"
path_out = f"{path_work}/sword2_PhageDepo_pt2"

with open(f"{path_project}/depolymerase_proteins.PhageDepo.txt", "r") as file:
    content = file.read()
    to_predic = ast.literal_eval(content)
    
to_do = []
for file in to_predic :
    if file not in os.listdir(path_out) :
        to_do.append(file)


***
# Identifying the domain delimitations

> First move the reps in the same file 

In [ ]:
path_orignin = "/home/conchae/PhageDepo_pdb/sword2_PhageDepo"
path_destination = "/home/conchae/PhageDepo_pdb/sword2_PhageDepo_pt2"

done_protein = ['3777', '1856', '2638', '1533', '3737', '1998', '3954', '2267', '1020', '202', '1436', '2800', '1502', '16', '4299', '3463', '2617', '1848', '1569', '2715', '397', '4042', '1620', '4088', '1857',
 '1858', '364', '1695', '1107', '3036', '3863', '1813', '4071', '396', '3792', '3274', '3425', '3456', '1755', '2624', '1779', '2944', '1424', '1106', '1091', '2038', '4143', '2963', '3947', '3940',
 '3754', '154', '3201', '1493', '2137', '1022', '2217', '4221', '2216', '3074', '4178', '3955', '2747', '3379', '3801', '2691', '4247', '4227', '3965', '2011', '3498', '2972', '402', '3442', '2540',
 '1503', '3296', '1517', '4248', '1420', '1717', '333', '403', '2531', '2962', '4477', '1203', '2700', '1516', '2596', '2732', '3720', '4211', '1965', '2709', '3651', '218', '1774', '369', '3174',
 '3909', '3930', '4338', '3767', '1966', '334', '1985', '4180', '4210', '4319', '3727', '29', '1531', '2714', '3683', '1918', '2632', '1842', '1344', '1463', '2243', '3799', '1727', '2956', '3126',
 '3603', '1994', '2602', '4472', '1143', '4302', '2908', '183', '2066', '1645', '4446', '4475', '1431', '2062', '2941', '1855', '3459', '1470', '2191', '226', '4261', '221', '3939', '2597', '2973',
 '4259', '2089', '4090', '2587', '428', '4230', '2464', '4120', '448', '294', '141', '3477', '2778', '3275', '3334', '2093', '3085', '4502', '3374', '2661', '2751', '134', '167', '2934', '1515',
 '1513', '1110', '1571', '1840', '2782', '3210', '2068', '3953', '3229', '2793', '3010', '3854', '2603', '159', '3113', '2892', '4549', '1570', '2291', '1847', '3684', '1147', '2598', '3208', '1568',
 '3239', '3317', '4497', '4089', '1454', '1527', '3329', '3392', '1952', '3056', '1566', '2012', '3800', '1536', '260', '2486', '361', '405', '2899', '2945', '3398', '4481', '1658', '2912', '1035',
 '1789', '3054', '3705', '1501', '2589', '3091', '4489', '3813', '3782', '2754', '4086', '2480', '3133', '425', '2765', '3532', '445', '1163', '2796', '3779', '144', '4036', '3906', '331', '4417',
 '3698', '4358', '1095', '1111', '3071', '1214', '3309', '3957', '1810', '2660', '3357', '1437', '1532', '3303', '4171', '3355', '1996', '164', '1112', '146', '3735', '1290', '3282', '2703', '4136',
 '3570', '3840', '4511', '4500', '184', '2749', '17', '3798', '1108', '3776', '163', '347', '1790', '2816', '3070', '373', '408', '4044', '2034', '2590', '4385', '3728', '3732', '3176', '3778',
 '1477', '2713', '1065', '2927', '2891', '1492', '233', '1661', '1923', '4186', '4160', '3276', '3164', '3597', '1563', '3595', '4215', '3112', '4070', '2002', '1869', '3750', '1711', '4263', '225']

import os 

for file in done_protein : 
    os.system(f"cp -r {path_orignin}/{file} {path_destination}")
    

In [ ]:
from multiprocessing.pool import ThreadPool
import subprocess
import os

path_project = "/home/conchae/PhageDepo_pdb"
path_pdb = f"{path_project}/sword2_PhageDepo_pt2"
path_db = "/home/conchae/depolymerase/RefDepo_domains/RefDepo_domains_db"
path_tmp = f"{path_project}/tmp"

def seek_beta_helix(path_in) :
    with open(f"{path_project}/scan_domains.ppt.log","a+") as outfile : 
        dir_out = f"{path_project}/seekfold_PhageDepo_domains"
        protein_id = path_in.split("/")[-1].split(".pdb")[0]
        path_out = f"{dir_out}/{protein_id}.out"
        output_frmt = f"query,target,pident,alnlen,gapopen,qstart,qend,tstart,tend,bits,prob"
        seek = f"foldseek easy-search {path_in} {path_db} {path_out} {path_tmp} --format-output {output_frmt}"
        seek_process = subprocess.Popen(seek, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        press_out, press_err = seek_process.communicate()
        print (press_out, press_err)
        outfile.write(f"{protein_id}\n")

paths_domain = []
for file in os.listdir(f"{path_pdb}") :
    try : 
        for file_2 in os.listdir(f"{path_pdb}/{file}/{file}_A/Protein_Units") :
            if file_2[-3:] == "pdb" :
                file_2_name = file_2.split(".pdb")[0]
                if os.path.isfile(f"{path_project}/seekfold_minibatch_domains/{file_2_name}.out") == False :
                    path = f"{path_pdb}/{file}/{file}_A/Protein_Units/{file_2}"
                    paths_domain.append(path)
    except Exception as e:
        print(file)

if __name__ == '__main__':
    with ThreadPool(30) as p:
        p.map(seek_beta_helix, paths_domain)
    
# ******************************************************************************************
#!/bin/bash
#BATCH --job-name=seek_PhageDepo__
#SBATCH --partition=medium 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=40
#SBATCH --mem=90gb 
#SBATCH --time=2-00:00:00 
#SBATCH --output=seek_PhageDepo__%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate HH-suite3

python /home/conchae/PhageDepo_pdb/script_files/seek_betahelix.phagedepo.py

***
## Seekfold on the full protein :



In [ ]:
from multiprocessing.pool import ThreadPool
import subprocess
import os

path_project = "/home/conchae/PhageDepo_pdb"
path_pdb = f"{path_project}/phagedepo_out"
path_prot = f"{path_project}/sword2_PhageDepo_pt2"
path_db = "/home/conchae/depolymerase/RefDepo_domains/RefDepo_domains_db"
path_tmp = f"{path_project}/tmp/tmp_ful"


def seek_beta_helix(path_in) :
    with open(f"{path_project}/scan_full.ppt.log","a+") as outfile : 
        dir_out = f"{path_project}/seekfold_PhageDepo_full"
        protein_id = path_in.split("/")[-1].split(".pdb")[0]
        path_out = f"{dir_out}/{protein_id}.out"
        output_frmt = f"query,target,pident,alnlen,gapopen,qstart,qend,tstart,tend,bits,prob"
        seek = f"foldseek easy-search {path_in} {path_db} {path_out} {path_tmp} --format-output {output_frmt}"
        seek_process = subprocess.Popen(seek, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        press_out, press_err = seek_process.communicate()
        print (press_out, press_err)
        outfile.write(f"{protein_id}\n")

paths_full = []
for file in os.listdir(f"{path_prot}") :
    path = f"{path_pdb}/{file}.pdb"
    paths_full.append(path)
    
if __name__ == '__main__':
    with ThreadPool(30) as p:
        p.map(seek_beta_helix, paths_full)
    
# ******************************************************************************************
#!/bin/bash
#BATCH --job-name=seek_PPT__
#SBATCH --partition=medium 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=40
#SBATCH --mem=90gb 
#SBATCH --time=2-00:00:00 
#SBATCH --output=seek_PPT__%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate HH-suite3

python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/script_files/seek_betahelix.full.ppt.py


In [ ]:
# Finish the sword2 process : 

#!/bin/bash
#BATCH --job-name=sword_PhageDepo__
#SBATCH --partition=short 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=20
#SBATCH --mem=90gb 
#SBATCH --time=1-00:00:00 
#SBATCH --output=sword_PhageDepo__%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate HH-suite3


/home/conchae/software/SWORD2/SWORD2.py \
-i /home/conchae/PhageDepo_pdb/phagedepo_out/6358.pdb \
-o /home/conchae/PhageDepo_pdb/sword2_PhageDepo_pt2/6358 \
-x 5 
 
/home/conchae/software/SWORD2/SWORD2.py \
-i /home/conchae/PhageDepo_pdb/phagedepo_out/786.pdb \
-o /home/conchae/PhageDepo_pdb/sword2_PhageDepo_pt2/786 \
-x 5 


/home/conchae/software/SWORD2/SWORD2.py \
-i /home/conchae/PhageDepo_pdb/phagedepo_out/51.pdb \
-o /home/conchae/PhageDepo_pdb/sword2_PhageDepo_pt2/51 \
-x 5 


> Screening the results 

In [ ]:
import sys
import os 
import pandas as pd 
from Bio import SeqIO

path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
dir_out_full = f"{path_project}/seekfold_out"
dir_out = f"{path_project}/rbp_work/seekfold_out_Depo77"
path_swords_out = f"{path_project}/rbp_work/sword2_depo"
path_info = f"/media/concha-eloko/Linux/depolymerase_building/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)
header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]

dico_folds_77 = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

dico_out = {}
for file in os.listdir(dir_out) :
    protein = file.split("_A")[0]
    if protein not in dico_out : 
        tmp_list = []
        tmp_list.append(f"{dir_out}/{file}")
        dico_out[protein] = tmp_list
    else :
        dico_out[protein].append(f"{dir_out}/{file}")

for protein in dico_out :
    results_full_prot = pd.read_csv(f"{dir_out_full}/{protein}.out" , sep = "\t" , names = header_seekfold) 
    for _,row in results_full_prot.iterrows():
        fold = info_df[info_df["ID"] == row["target"]]["X_Group_Name"].values[0]
        if fold != "jelly-roll" :
            full_longest = row["alnlen"]
            full_score = row["bits"]
            full_prob = row["prob"]
            full_len = len(list(SeqIO.parse(f"{path_swords_out}/{protein}/{protein}_A/SWORD/{protein}_A/{protein}_A.fasta", "fasta"))[0])
            full_metric = (full_prob) * (full_score**0.4) * (full_longest**0.8) / (full_len**0.3)
            b = (full_longest , full_len , round(full_prob,4) , round(full_metric,4))
            break
        # the winner, tuple with 4 elements : 
    # (the aln length, the PU length, the bitscore, the name)
    final_boss = (full_metric ,results_full_prot.loc[0,"query"] ,f"Full protein {full_len}")
    the_winner = (0 , 0)
    for path in dico_out[protein] :
        if os.path.getsize(path) > 0 :
            results_df = pd.read_csv(f"{path}", sep = "\t" , names = header_seekfold)
            fold = info_df[info_df["ID"] == results_df.loc[0, "target"]]["X_Group_Name"].values[0]
            if fold != "jelly-roll" :
                aln_longest = int(results_df["alnlen"].max())
                aa_end = results_df.loc[0, "query"].split("_")[-1].split(".pdb")[0]
                aa_start = results_df.loc[0, "query"].split("_")[-2]
                PU_length = int(aa_end) - int(aa_start)
                score = results_df.loc[0, "bits"]
                prob = results_df.loc[0, "prob"]
                #final_metric = (prob) * (score**0.4) * (aln_longest**0.8) / (PU_length**0.3)
                final_metric = (prob) * (score**0.4) * (aln_longest**0.95) / (PU_length**0.3)
                contender = (final_metric , results_df.loc[0, "query"])
                a = (aln_longest , PU_length ,score, round(prob,4) , round(final_metric,4), results_df.loc[0, "query"], fold)
                if final_metric > 10 :
                    print(a)
                if contender[0] > the_winner[0] :
                    the_winner = contender
                else :
                    continue
    if the_winner[0] < final_boss[0] :
        the_winner = final_boss
    else :
        pass
    print("And the winner is : \n",the_winner)
    print("Here was the final boss :", b )
    print("\n")

        
